In [3]:
from ipywidgets import widgets
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure

import numpy as np
import nibabel as nib

In [ ]:
hippo_left = nib.load('../data/data/data/train_freesurfers_preprocess/OAS30007_MR_d0061/posterior_Left-Hippocampus.nii.gz')
hippo_right = nib.load('../data/data/data/train_freesurfers_preprocess/OAS30007_MR_d0061/posterior_Right-Hippocampus.nii.gz')

In [15]:
hippo_left_true = nib.load('../data/try/OAS30007_MR_d0061/mri/hippocampus_left.nii.gz')
hippo_right_true = nib.load('../data/try/OAS30007_MR_d0061/mri/hippocampus_right.nii.gz')

In [17]:
hippo_left_true.get_fdata().shape

(1, 40, 40, 40)

In [20]:
hippo1 = np.concatenate((hippo_left_true.get_fdata()[0, :, :, :], hippo_right_true.get_fdata()[0, :, :, :]))
hippo2 = np.concatenate((hippo_left.get_fdata(), hippo_right.get_fdata()))

In [22]:
@widgets.interact_manual(
    hippo_type=['Freesurfer', 'Fastsurfer']
)
def plot_3d_plotly_shape(hippo_type, show_mesh=True, crop=True, grid=True):
  import plotly.graph_objects as go

  if hippo_type == 'Freesurfer':
    hippo = hippo2
  else:
    hippo = hippo1

  vert_p, faces_p, normals_p, values_p = measure.marching_cubes(hippo, 0, spacing=(1, 1, 1))

  fig = go.Figure(data=[go.Mesh3d(
        x=vert_p[:, 0],
        y=vert_p[:, 1],
        z=vert_p[:, 2],
        # i, j and k give the vertices of triangles
        # here we represent the 4 triangles of the tetrahedron surface
        i=faces_p[:, 0],
        j=faces_p[:, 1],
        k=faces_p[:, 2],
        name='y',
        showscale=True
    )]
  )

  if show_mesh:
    #plot surface triangulation
    tri_vertices = vert_p[faces_p]
    Xe = []
    Ye = []
    Ze = []
    
    for T in tri_vertices:
      Xe += [T[k%3][0] for k in range(4)] + [ None]
      Ye += [T[k%3][1] for k in range(4)] + [ None]
      Ze += [T[k%3][2] for k in range(4)] + [ None]
       
    fig.add_trace(go.Scatter3d(
                     x=Xe,
                     y=Ye,
                     z=Ze,
                     mode='lines',
                     name='',
                     line=dict(color= 'rgb(40,40,40)', width=0.5)))
  if crop:
    scale_min = np.min(vert_p, axis=0)
    scale_max = np.max(vert_p, axis=0)
  else:
    scale_min = [0, 0, 0]
    scale_max = pred_data.shape
      
  fig.update_layout(
    scene = dict(aspectratio=dict(x=1, y=1, z=1),
        xaxis = dict(range=[scale_min[0], scale_max[0]], visible=grid),
        yaxis = dict(range=[scale_min[1], scale_max[1]], visible=grid),
        zaxis = dict(range=[scale_min[2], scale_max[2]], visible=grid),) 
    )

  fig.show()

interactive(children=(Dropdown(description='hippo_type', options=('Freesurfer', 'Fastsurfer'), value='Freesurf…